## Introduction
In the previous post, we read about the concepts of __Graph__ and __Session__ which describes the way the data flows in TensorFlow. In this tutorial, we'll take a look at some of the __Tensor Types__ used in TensorFlow and specially the ones commonly used in creating neural network models, namely ___Constant___, ___Variable___, and ___Placeholder___. 

This will also enable us to shed light on some of the points and questions left unanswered in the previous post.

Remember that we need to import the TensorFlow library at the very beginning of our code using:

In [2]:
import tensorflow as tf




## 1. Constant 

As the name speaks for itself, __Constants__ are used as constant value tensors. They create a node that takes a constant value. You can simply create a constant tensor using __tf.constant__. It accepts the following arguments:


In [ ]:
tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)

Now let's look at a very simple example.

### Example 1:
Let's create two constants and add them together. Constant tensors can be defined simply by defining a value:

In [ ]:
# create graph
a = tf.constant(2)
b = tf.constant(3)
c = a + b
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(c))

Perfect! Now Let's look at the created graph and generated data types:
<img src="files/files/2_1.png" width="1000" height="2000" >
___Fig1. ___ __Left:__ generated graph visualized in Tensorboard, __Right:__ generated variables (screenshot captured from PyCharm debugger when running in debug mode)

As it's depicted in the figure, we created 3 tensors with __"Python-names"__ _a_, _b_, and _c_. However, we didn't define any __"TensorFlow-name"__ for them. Therefore, TensorFlow assigns some default names to them which are depicted in the graph: __const__ and __const_1__ for the input constants and __add__ for the output of the addition operation. We can easily modify it and define our own names, like:

In [ ]:
# create graph
a = tf.constant(2, name='A')
b = tf.constant(3, name='B')
c = tf.add(a, b, name='Sum')
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(c))

This time the graph and created tensors are as follows:
<img src="files/files/2_2.png" width="1000" height="2000" >
___Fig2. ___ generated graph (Left) and variables (Right) with the modified names


We can also define constants of different types (integer, float, etc.) and shapes (vectors, matrices, etc.).


### Example 2:

In [ ]:
s = tf.constant(2.3, name='scalar', dtype=tf.float32)
m = tf.constant([[1, 2], [3, 4]], name='matrix')
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(s))
    print(sess.run(m))

## 2. VARIABLE
Variables are stateful nodes which output their current value; meaning that they can retain their value over multiple executions of a graph. They have a number of useful features such as:

- They can be __saved__ to your disk during and after training. This allows people from different companies and groups to save, restore and send over their model parameters to other people.
- By default, gradient updates (used in all neural networks) will apply to all variables in your graph. In fact, variables are the things that you want to tune in order to minimize the loss. 

These features make variables suitable to be used as the network parameters (i.e. weights and biases).

You might ask what are the differences between variables and constants? Well there are two major differences:

1. Constants are (guess what), constants. Their value doesn't change. You'd usually need your network parameters to be updated and that's where the __variable__ comes into play.

2. Constants are stored in the graph definition which makes them memory-expensive. In other words, constants with millions of entries makes the graph loading much slower.


Again, it's important to remember that creating a variables is an operation (look at the Fig. 2 of the first tutorial for a quick recap). When we evaluate these operations in the session, we'll get the output value of the operations.

### 2.1. Create Variables
To create a variable, you can use __tf.Variable__ as:


In [ ]:
# Create a variable.
w = tf.Variable(<initial-value>, name=<optional-name>)

Some examples of creating scalar and matrix variables are as follows:

In [ ]:
s = tf.Variable(2, name="scalar") 
m = tf.Variable([[1, 2], [3, 4]], name="matrix") 
W = tf.Variable(tf.zeros([784,10]))

Variable __W__ defined above will create a matrix with 784 rows and 10 columns which will be initialized by zeros. This can be used as a weight matrix of a feed-forward neural network (or even in a linear regression model) from a layer with 784 neuron to a layer with 10 neuron. We'll see more of this later in this turorial.

__*Note:__ we use tf.Variable() with uppercase "V", and tf.constant with lowercase "c". You don't necessarily need to know the reason, but it's simply because tf.constant is an op, while tf.Variable is a class with multiple ops.

__* IMPORTANT Note:__ Calling tf.Variable to create a variable is the old way of creating a variable. TensorFlow recommends to use the wraper __tf.get_variable__ which accepts the name, shape, etc as its arguments as follows:


In [ ]:
tf.get_variable(name,
                shape=None,
                dtype=None,
                initializer=None,
                regularizer=None,
                trainable=True,
                collections=None,
                caching_device=None,
                partitioner=None,
                validate_shape=True,
                use_resource=None,
                custom_getter=None,
                constraint=None)

Some examples are as follows:

In [ ]:
s = tf.get_variable("scalar", initializer=tf.constant(2)) 
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.get_variable("weight_matrix", shape=(784, 10), initializer=tf.zeros_initializer())

### 2.2. Initialize Variables
Variables need to be initialized befor being used. To do so, we have to invoke a __variable initializer operation__ and run the operation on the session. This is the easiest way to initialize variables which initializes all variables at once.

The following toy example shows how we can add an op to initialize the variables.

### Example 3:
create two variables and add them together. Then print out their value and the summation result.

In [ ]:
a = tf.get_variable(name="var_1", initializer=tf.constant(2))
b = tf.get_variable(name="var_2", initializer=tf.constant(3))
c = tf.add(a, b, name="Add1")

# launch the graph in a session
with tf.Session() as sess:
    # now let's evaluate their value
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

As you can see, we ran into __FailedPreconditionError: Attempting to use uninitialized value__. This is because we tried to evaluate the variables before initializing them. Let's correct the code by first initializing all variables and then evaluating them.

In [3]:
# create graph
a = tf.get_variable(name="A", initializer=tf.constant(2))
b = tf.get_variable(name="B", initializer=tf.constant(3))
c = tf.add(a, b, name="Add")
# add an Op to initialize global variables
init_op = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # run the variable initializer operation
    sess.run(init_op)
    # now let's evaluate their value
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

2
3
5


Let's take a quick look at the graph and generated variables:
<img src="files/files/2_3.png" width="1000" height="2000" >
___Fig3. ___ generated graph (Left) and variables (Right)

As you can see, two blue boxes are the generated variables (compare them with constant nodes in Fig. 2) which are added together using "Add" operation.


__*Note:__ Variables are usually used for weights and biases in neural networks.

- __weights__ are usually initialized from a normal distribution using `tf.truncated_normal_initializer()`.

- __biases__ are usually initialized from zeros using `tf.zeros_initializer()`.

Let's look at a very simple example of creating weight and bias variables with proper initialization:

### Example 4:
Create the weight and bias matrices for a fully-connected layer with 2 neuron to another layer with 3 neuron.
In this scenario, the weight and bias variables must be of size $[2, 3]$ and 3 respectively.

In [4]:
# create graph
weights = tf.get_variable(name="W", shape=[2,3], initializer=tf.truncated_normal_initializer(stddev=0.01))
biases = tf.get_variable(name="b", shape=[3], initializer=tf.zeros_initializer())

# add an Op to initialize global variables
init_op = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # run the variable initializer
    sess.run(init_op)
    # now we can run our operations
    W, b = sess.run([weights, biases])
    print('weights = {}'.format(W))
    print('biases = {}'.format(b))

ValueError: Variable W already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-3-531aeccfa637>", line 2, in <module>
    weights = tf.get_variable(name="W", shape=[2,3], initializer=tf.truncated_normal_initializer(stddev=0.01))
  File "/home/cougarnet.uh.edu/amobiny/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/cougarnet.uh.edu/amobiny/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2822, in run_ast_nodes
    if self.run_code(code, result):


## 3. Placeholder:
Placeholders are nodes whose value is fed in at execution time. If you have inputs to your network that depend on some external data and you don't want your graph to depend on any real value, placeholders are the datatype you need. In fact, you can build the graph without needing the data. Therefore, they don't need any initial value; only a datatype (such as float32) and a tensor shape so the graph still knows what to compute even though it doesn't have any stored values yet.

Some examples of creating placeholders are as follows:

In [ ]:
a = tf.placeholder(tf.float32, shape=[5])
b = tf.placeholder(dtype=tf.float32, shape=None, name=None)
X = tf.placeholder(tf.float32, shape=[None, 784], name='input')
Y = tf.placeholder(tf.float32, shape=[None, 10], name='label') 

Let's run a simple example.

### Example 5:
Create a constant vector and a placeholder and add them together.

In [5]:
a = tf.constant([5, 5, 5], tf.float32, name='A')
b = tf.placeholder(tf.float32, shape=[3], name='B')
c = tf.add(a, b, name="Add")

with tf.Session() as sess:
      print(sess.run(c))

InvalidArgumentError: You must feed a value for placeholder tensor 'B' with dtype float and shape [3]
	 [[Node: B = Placeholder[dtype=DT_FLOAT, shape=[3], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: Add/_1 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7_Add", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op u'B', defined at:
  File "/home/cougarnet.uh.edu/amobiny/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/cougarnet.uh.edu/amobiny/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/cougarnet.uh.edu/amobiny/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/cougarnet

As you can see, running this code will run into __InvalidArgumentError: You must feed a value for placeholder tensor 'B' with dtype float and shape $[3]$__ error. As you might have guessed, it is simply because the placeholder is empty and there is no way to add an empty tensor to a constant tensor. To solve this, we need to feed the input value to tensor "a". It can be done by creating a dictionary ("d" in the following code) whose key(s) are the placeholders and their values are the desired value to be passed to the placeholder(s), and feeding it to an argument called "feed_dict". In our example, say we want to pass $[1, 2, 3]$ to the placeholder; the code needs to be modified as:


In [6]:
a = tf.constant([5, 5, 5], tf.float32, name='A')
b = tf.placeholder(tf.float32, shape=[3], name='B')
c = tf.add(a, b, name="Add")

with tf.Session() as sess:
    # create a dictionary:
    d = {b: [1, 2, 3]}
    # feed it to the placeholder
    print(sess.run(c, feed_dict=d)) 

[6. 7. 8.]


The generated graph and variables are as follows:
<img src="files/files/2_4.png" width="700" height="1400" >
___Fig4. ___ generated graph (Left) and variables (Right)



So far so good?


## Creating a Toy Neural Network
Now, we have all the required materials to start building a toy feed-forward neural network with one hidden layer with 200 hidden units (neurons). The computational graph in Tensorflow will look like this:

<img src="files/files/2_5.png" width="300" height="600" >
___Fig5. ___ Schematic of the graph for one layer of the neural network

How many operations (or nodes) you see in this graph? Six, right? The three circles (X, W, b) and three rectangles. We'll go through them one by one and will discuss what is the best way to implement it.

Let's start with the input, X. This can be an input of any type, such as images, signals, etc. The general approach is to feed all inputs to the network and train the trainable parameters (here, W and b) by backpropagating the error signal. Ideally, you need to feed all inputs together, compute the error, and update the parameters. This process is called "Gradient Descent".

*Side Note: In real-world problems, we have thousands and millions of inputs which makes gradient descent computationally expensive. That's why we split the input set into several shorter pieces (called mini-batch) of size B (called mini-batch size) inputs, and feed them one by one. This is called "Stochastic Gradient Descent". The process of feeding each mini-batch of size B to the network, back-propagating errors, and updating the parameters (weights and biases) is called an iteration.

We generally use Placeholders for inputs so that we can build the graph without depending on any real value. The only point is that you need to choose the proper size for the input. Here, we have a feed-forward neural network, and let's assume inputs of size 784 (similar to 28x28 images of MNIST data). The input placeholder can be written as:

In [ ]:
# create the input placeholder
X = tf.placeholder(tf.float32, shape=[None, 784], name="X")

You might wonder what is shape=$[None, 784]$?!

Well, that's the tricky part! Read the above side note one more time. We need to feed B images of size 784 to the network in each training iteration. So the placeholder needs to be of shape=$[B, 784]$. Defining the placeholder shape as $[None, 784]$ means that you can feed any number of images of size 784 (not B images necessarily). This is especially helpful in the evaluation time where you need to feed all validation or test images to the network and compute the performance on all of them.

Enough with the placeholder. Let's continue with the network parameters, W, and b. As explained in the Variable section above, they have to be defined as variables. Since in Tensorflow, gradient updates will be applied to the graph variables, by default. As mentioned, variables need to be initialized.

*Note: Generally, weights (W) are initialized randomly, in it's the simplest form from a normal distribution, say normal distribution with zero mean and standard deviation of 0.01. Biases (b) can be initialized as small constant values, such as 0.

Since the input dimension is 784 and we have 200 hidden units, the weight matrix will be of size $[784, 200]$. We also need 200 biases, one for each hidden unit. The code will be like:

In [ ]:
# create weight matrix initialized randomely from N(0, 0.01)
weight_initer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)
W = tf.get_variable(name="W", dtype=tf.float32, shape=[784, 200], initializer=weight_initer)

# create bias vector of size 200, all initialized as zero
bias_initer =tf.constant(0., shape=[200], dtype=tf.float32)
b = tf.get_variable(name="b", dtype=tf.float32, initializer=bias_initer)

Now let's move on to the rectangle operations. We must multiply input X$_{[None, 784]}$ and weight matrix W$_{[784, 200]}$ which gives a tensor of size $[None, 200]$, then add the bias vector b$_{[200]}$ and eventually pass the final tensor from a ReLU non-linearity:

In [ ]:
# create MatMul node
x_w = tf.matmul(X, W, name="MatMul")
# create Add node
x_w_b = tf.add(x_w, b, name="Add")
# create ReLU node
h = tf.nn.relu(x_w_b, name="ReLU")
 